In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from copy import deepcopy
from matplotlib import pyplot as plt

import os
import datetime
import time
import json
import pickle
import os

NAFILL = -999

In [2]:
with open("config.json", "r") as f:
    config = json.load(f)
folder_path = config["data_root_path"]
focus_path = folder_path + "Selection_Group/" + config["label_focus"] + "/"
epis = config["epis"]
wks = config["wks"]

In [3]:
with open(focus_path + "raw_data_pool.pkl", "rb") as f:
    raw_data_pool = pickle.load(f)
df_phaseI_select = raw_data_pool["data_phaseI"]
df_phaseII_select = raw_data_pool["data_phaseII"]
df_label = raw_data_pool["labels"]

In [4]:
df_phaseI_select.shape

(7950, 2115)

In [13]:
with open(folder_path + "feature_col_epis.json","r") as f:
    feature_col_epis = json.load(f)

In [16]:
df_wks_epis = {}
for wk in wks:
    if wk not in df_wks_epis:
        df_wks_epis[wk] = {}
    for epi in epis:
        print(wk,epi)
        df_buf = df_phaseI_select[df_phaseI_select["wk"] == wk][["PID"] + feature_col_epis[epi]]
        if(df_buf.shape[1] == 1): continue
        df_buf_mean = df_buf.groupby("PID").mean()
        df_buf_mean.columns = ["mean_" + x for x in df_buf_mean.columns]
        df_buf_std = df_buf.groupby("PID").std()
        df_buf_std.columns = ["std_" + x for x in df_buf_std.columns]
        df_buf_mean_std = df_buf_mean.merge(df_buf_std, how = "outer", left_index = True, right_index = True)
#         df_buf_mean_std = (df_buf_mean_std-df_buf_mean_std.mean())/df_buf_mean_std.std()
        df_wks_epis[wk][epi] = deepcopy(df_buf_mean_std)

wkdy mo
wkdy af
wkdy ev
wkdy ni
wkend mo
wkend af
wkend ev
wkend ni


In [90]:
from sklearn.feature_selection import mutual_info_classif

In [227]:
top_features_ig = {}
for wk in wks:
    top_features_ig[wk] = {}
    for epi in epis:
        Y = df_label.set_index("PID")["label_I"]
        X = df_wks_eps[wk][epi].loc[Y.index]
        ig = dict(zip(X.columns, mutual_info_classif(X.fillna(NAFILL), Y, discrete_features = False)))
        ig = [(k, ig[k]) for k in sorted(ig, key=ig.get, reverse = True)]
        top_info_gain = [x[0] for x in ig[:100]]
        top_features_ig[wk][epi] = top_info_gain
overlap_features = top_features_ig
overlap_features_remain_ = deepcopy(overlap_features)
for i in range(30):
    features_ig = {}
    for wk in wks:
        features_ig[wk] = {}
        for epi in epis:
            Y = df_label.set_index("PID")["label_I"]
            X = df_wks_eps[wk][epi].loc[Y.index]
            ig = dict(zip(X.columns, mutual_info_classif(X.fillna(NAFILL), Y, discrete_features = False)))
            ig = [(k, ig[k]) for k in sorted(ig, key=ig.get, reverse = True)]
            top_features_ig = [x[0] for x in ig[:100]]
            overlap_features_remain_[wk][epi] = list(set(overlap_features_remain_[wk][epi]).intersection(set(top_features_ig)))

    for wk in wks:
        for epi in epis:
            print(len(overlap_features_remain_[wk][epi]),end = " ")
    print()


67 65 76 70 62 66 69 65 
58 53 69 59 50 55 58 55 
54 45 65 51 48 49 56 49 
53 44 63 47 47 47 55 48 
53 43 60 45 46 42 54 45 
52 41 60 45 44 41 53 44 
52 41 59 43 43 40 50 44 
52 41 59 42 43 40 50 44 
52 41 59 41 43 40 50 43 
51 41 58 41 43 40 49 43 
51 41 57 41 43 39 49 43 
51 41 56 41 42 39 49 43 
50 41 56 41 42 39 49 43 
50 40 54 41 41 37 48 42 
50 40 54 41 41 37 48 41 
50 40 54 41 41 37 48 41 
50 40 54 41 41 36 48 41 
50 40 54 41 40 35 48 41 
50 40 53 41 40 35 48 41 
49 40 53 40 40 34 48 41 
49 40 53 39 38 34 48 41 
49 39 52 39 38 34 48 41 
49 39 52 39 38 34 48 41 
49 39 52 39 38 34 48 41 
49 39 52 39 38 34 48 41 
48 39 52 39 38 34 48 41 
48 39 52 39 38 34 48 41 
48 39 52 38 38 33 48 41 
48 39 52 38 38 32 48 41 
48 39 52 38 38 32 47 41 


In [5]:
a = [19,20,19,16,20,15,21,18]

In [6]:
print(np.mean(a), np.sum(a), np.min(a), np.max(a))

18.5 148 15 21


In [ ]:
with open(focus_path + "top_features_raw.json", "w") as f:
    json.dump(overlap_features_remain_,f)

In [9]:
with open(focus_path + "top_features_raw.json", "r") as f:
    overlap_features_remain_ = json.load(f)

In [10]:
from collections import Counter

In [18]:
overlap_features_remain_filtered = deepcopy(overlap_features_remain_)
for wk in wks:
    for epi in epis:
        print(wk,epi)
        df_buf = df_wks_epis[wk][epi][overlap_features_remain_filtered[wk][epi]]
        df_buf_corr = df_buf.corr()
        upper = df_buf_corr.where(np.triu(np.ones(df_buf_corr.shape), k=1).astype(np.bool)).abs()
        max_corr = upper.max().max()
        while (max_corr > 0.5):
            row_idxmax = upper.max(axis = 1).idxmax()
            col_idxmax = upper.max(axis = 0).idxmax()
            row_cor_sum = df_buf_corr.abs()[row_idxmax].sum()
            col_cor_sum = df_buf_corr.abs()[col_idxmax].sum()
            if(row_cor_sum > col_cor_sum):
                overlap_features_remain_filtered[wk][epi].remove(row_idxmax)
                print("remove ", row_idxmax)
            else:
                overlap_features_remain_filtered[wk][epi].remove(col_idxmax)
                print("remove ", col_idxmax)
            df_buf = df_wks_epis[wk][epi][overlap_features_remain_filtered[wk][epi]]
            df_buf_corr = df_buf.corr()
            upper = df_buf_corr.where(np.triu(np.ones(df_buf_corr.shape), k=1).astype(np.bool)).abs()
            max_corr = upper.max().max()
#         plt.matshow(df_buf_corr)
#         plt.show()
        print(Counter([x.split("_")[2] for x in overlap_features_remain_filtered[wk][epi]]))


wkdy mo
remove  mean_f_locMap_num_bouts_at_place_006_morning
remove  std_f_locMap_min_bout_at_place_007_morning
remove  std_f_blue_avg_num_scans_of_all_devices_of_self_morning
remove  std_f_locMap_mean_bout_at_place_007_morning
remove  std_f_locMap_min_bout_at_place_outside_morning
remove  std_f_blue_number_samples_bluetooth_morning
remove  std_f_locMap_mean_bout_at_place_outside_morning
remove  mean_f_blue_sum_num_scans_of_all_devices_of_others_morning
remove  mean_f_locMap_mean_bout_at_place_outside_morning
remove  mean_f_locMap_max_bout_at_place_outside_morning
remove  mean_f_screen_max_len_minute_interaction_bout_morning
remove  mean_f_screen_std_len_minute_interaction_bout_morning
remove  std_f_screen_max_len_minute_interaction_bout_morning
remove  mean_f_locMap_num_bouts_20min_or_more_at_place_outside_morning
remove  std_f_locMap_max_bout_at_place_007_morning
remove  mean_f_locMap_min_bout_at_place_outside_morning
remove  mean_f_loc_moving_time_percent_local_morning
remove  mean_

In [20]:
overlap_features_remain = {}
for wk in wks:
    overlap_features_remain[wk] = {}
    for epi in epis:
        print(len(overlap_features_remain_filtered[wk][epi]), end = ",")
        overlap_features_remain[wk][epi] = deepcopy(list(set(["_".join(x.split("_")[1:]) for x in overlap_features_remain_filtered[wk][epi]])))
        print(len(overlap_features_remain[wk][epi]))

19,19
20,20
20,19
16,16
20,20
15,15
21,21
18,18


In [263]:
with open(focus_path + "top_features.json", "w") as f:
    json.dump(overlap_features_remain,f)

In [264]:
top_features_dis = {}
for wk in wks:
    top_features_dis[wk] = {}
    for ep in epis:
        for f in overlap_features_remain[wk][ep]:
            split_f = f.split("_")
            f_dis = "_".join(split_f[:-1] + ["dis"] + split_f[-1:])
            if (ep not in top_features_dis[wk]):
                top_features_dis[wk][ep] = [f_dis]
            else:
                top_features_dis[wk][ep].append(f_dis)

In [265]:
with open(focus_path + "top_features_dis.json", "w") as f:
    json.dump(top_features_dis,f)